In [9]:
import requests
import pandas as pd
from IPython.display import display
import re

API_KEY = "01946b8515c545443cdcd262a884a88dab1be54962aad37f4f93c3420cc49844"
headers = {"X-API-Key": API_KEY}

def extract_data (urls): 

    params = {
        "limit": 1000,
        "page": 1
    }

    all_results = []

    while True:
        response = requests.get(urls, headers=headers, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        results = data.get('results', [])
        all_results.extend(results)

        if len(results) < params['limit']:
            break

        params['page'] += 1

        if params['page'] > 1:  # Stop after fetching 3 pages
            break

    
    print(f"Total records retrieved: {len(all_results)}")
    print(f'extract_output {all_results[:3]}')
    return all_results 


# Getting worldwide sensor locations and unpack the country
url= "https://api.openaq.org/v3/locations"
locations_data = extract_data(url)
df_locations = pd.DataFrame(locations_data)
print(f'df_locations raw content {df_locations.head(3)}')
df_country = df_locations['country'].apply(pd.Series)
df_country.columns= ['country_id', 'country_code', 'country_name']
df_locations=pd.concat([df_locations,df_country],axis=1)
df_locations.drop(['owner', 'provider', 'isMobile', 'isMonitor', 'instruments', 'licenses', 'bounds', 'distance', 'country'], axis=1, inplace=True)
df_locations.rename(columns={'id': 'location_id', 'name': 'location_name'}, inplace=True)
print(f'df_location clean output : {df_locations[:3]}')
df_locations.to_csv('pollution_data/locations_clean1502.csv')

# get the sensor ID list  along with the location id

location_sensor_pairs = [(location["id"], sensor["id"]) 
                         for location in locations_data 
                         for sensor in location["sensors"]]
print(f'locations_sensors_pairs_list output :  {location_sensor_pairs[:5]}')

# generate URLs list
urls_list = [f"https://api.openaq.org/v3/sensors/{pair[1]}" for pair in location_sensor_pairs]
print(f'url_list {urls_list[:3]}')

# generate sensor data  dataframe

data_results = []
for url in urls_list[:3] : 
    sensors_data = extract_data(url)
    data_results.extend(sensors_data)
    data_df = pd.DataFrame(data_results)
    print (f'data df output:{data_df[:3]}')
    print (f'data df output legnth:{len(data_df)}')
    


# generate the location id list and dataframe 
sensor_id_list= [location[0] for location in  location_sensor_pairs]
print(sensor_id_list[:3])
sensor_id_df = pd.DataFrame(sensor_id_list)
print(f'sensor_id_df {sensor_id_df.head(3)}')

#conacetenate sensor data with location ID
final_df = pd.concat([sensor_id_df, data_df], axis=1)
print (f'final df:{final_df[:3]}')
print (f'final df legnth:{len(final_df)}')

Total records retrieved: 1000
extract_output [{'id': 3, 'name': 'NMA - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 209, 'name': 'Dr. Raphael E. Arku and Colleagues'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 6, 'name': 'pm10 µg/m³', 'parameter': {'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10'}}, {'id': 5, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], 'coordinates': {'latitude': 5.58389, 'longitude': -0.19968}, 'licenses': None, 'bounds': [-0.19968, 5.58389, -0.19968, 5.58389], 'distance': None, 'datetimeFirst': None, 'datetimeLast': None}, {'id': 4, 'name': 'NMT - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, 'n

In [13]:



# Extraire le numéro de sensor depuis l'URL
#sensor_id_match = re.search(r'/sensors/(\d+)', url)
#sensor_id = int(sensor_id_match.group(1)) if sensor_id_match else None

# Ajouter la colonne 'sensor_id' au DataFrame
#df_sensor['sensor_id'] = sensor_id

# Afficher le DataFrame
#display(df_sensor)



#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url_1 = "https://api.openaq.org/v3/sensors/3917"
#url_2 = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/locations?bbox=-10.5,36,9.6,51.1"
#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url = "https://api.openaq.org/v3/sensors/3917/years"
#url = "https://api.openaq.org/v3/sensors/3917/days"
#url = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/parameters"
#url = "https://api.openaq.org/v3/locations"
#url = "https://api.openaq.org/v3/countries"
